In [16]:
import pandas as pd
import numpy as np
from pathlib import Path

In [17]:
from os import walk

filenames = next(walk("/share/data/school_choice/Data"), (None, None, []))  # [] if no file

In [18]:
filenames[2]

['Max Reported Capacity.xlsx',
 'out_SFUSD20191220_1_1_sas_1819r3_prerun_student.dta',
 'test_frl_data.csv',
 '2018-19 SAS Schools Master List.xlsx',
 'High Level Overview of SAS_October 11 2017.docx',
 'est FRL by Census Tract (file shared 20191218).csv',
 'SF 2010 blks 022119 with field descriptions (1).xlsx',
 'program_codes.csv',
 '.DS_Store',
 'SAS Pre- and Post-Run File Specifications 2018_05_23 (1).xlsx',
 'aa2aa_migration_master_workbook.xlsx',
 'Elementary School Programs (Enrollment Guide).xlsx',
 'program_codes.xlsx',
 'High Level Overview of SAS_October 11 2017updated.docx',
 'test_frl_metrics.csv']

In [19]:
student_df = pd.read_excel("/share/data/school_choice/Data/SF 2010 blks 022119 with field descriptions (1).xlsx")

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [4]:
student_df

,Unnamed: 0,studentno,r1_ranked_idschool,r1_listed_ranks,r1_programs,grade,r1_randomnumber,randomnumber,requestprogramdesignation,latitude,...,AALPI Score,Academic Score,N'hood SES Score,FRL Score,sibling,currentlpsibling,currentlp,msf,aaprek,aa
0,0,10019435,"[815, 697, 559]","[1, 2, 3]","['CA', 'CA', 'CA']",13,"[0.8926267881169321, 0.5997281950450141, 0.095...",0.298533,0.0,37.736967,...,0.204972,0.101064,0.387407,0.066313,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10031302,"[815, 559, 439, 559, 785]","[1, 2, 3, 4, 5]","['CA', 'BA', 'CA', 'CA', 'CA']",13,"[0.40869254941208505, 0.259257627732611, 0.231...",0.006416,0.0,37.708783,...,0.517824,0.490157,0.629132,0.619612,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10033269,[764],[1],['CA'],13,[0.996115733080247],0.229133,0.0,37.734997,...,0.754253,0.938095,0.932494,0.822137,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10034607,"[815, 853]","[1, 2]","['CA', 'CA']",13,"[0.346680243635614, 0.013923836320173]",0.373381,0.0,37.709339,...,0.411856,0.417219,0.684535,0.671444,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10034799,"[559, 725, 785]","[1, 2, 3]","['CA', 'CA', 'CA']",13,"[0.386016426419183, 0.841100588083223, 0.01709...",0.424068,0.0,37.781808,...,0.560976,0.800353,0.815533,0.840754,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15569,15569,888042949,NaN,NaN,NaN,3,NaN,0.616349,0.0,37.732712,...,0.865119,0.945841,0.912961,0.789762,NaN,NaN,NaN,NaN,NaN,NaN
15570,15570,888042967,NaN,NaN,NaN,1,NaN,0.771211,1.0,-25.392681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[481]
15571,15571,888042990,NaN,NaN,NaN,3,NaN,0.495014,0.0,37.779201,...,0.065997,0.203518,0.512252,0.412294,NaN,NaN,NaN,NaN,NaN,[664]
15572,15572,888042991,NaN,NaN,NaN,6,NaN,0.277164,0.0,37.572334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[634],NaN,NaN
